<a href="https://colab.research.google.com/github/jorgel-mendes/HackDataSantanderIPYNB/blob/master/Hackathon_investiga%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rumo ao Prospper

Esse notebook mostra exploração de dados que embasou a criação do Prospper, uma solução integrada com o CoPiloto Santander que busca empoderar donos de negócio fornecendo informação em um formato de simples utilização. 

A solução foi criada para o Hackathon Data Challenge do Santander do grupo 41.

Na nossa análise de compreensão do problema utilizamos dados do IBGE recolhidos no site do [IPEA](http://www.ipeadata.gov.br/Default.aspx) e uma [API em Python](https://github.com/GeneralMills/pytrends) para o google trends.

In [1]:
!pip install -q pytrends

In [2]:
import pandas as pd
import altair as alt
alt.renderers.enable(embed_options={'theme': 'googlecharts'})

from datetime import datetime

## Análise do setor de Varejo em 2020



Pensando em como a crise afetou os pequenos negócios em geral, decidimos focar no setor de varejo.

Foi utilizada a base de Indíce de varejo do IBGE para acompanhar o desempenho do setor nos ultimos anos. Os dados da base foram colhidos no IPEADATA, site do Instituto de Pesquisa Econômica Aplicada (IPEA).

In [3]:
ipea_varejo = pd.read_csv('https://raw.githubusercontent.com/jorgel-mendes/HackDataSantanderIPYNB/master/ipeadata%5B02-08-2020-01-36%5D.csv')
ipea_varejo.drop(ipea_varejo.columns[2], axis=1, inplace=True)
ipea_varejo.columns = ['Data', 'Vendas reais (varejo)']
ipea_varejo['Data'] = ipea_varejo['Data'].apply(
    lambda x: datetime.strptime(str(x), '%Y.%m'))

alt.Chart(
    ipea_varejo.query('Data > "2008-12-31"')
    ).mark_line().encode(
    x=alt.X('Data', title=''),
    y=alt.Y('Vendas reais (varejo)',
            title='Index de vendas de varejo',
            scale=alt.Scale(zero=False))
).properties(
    title="Em abril o index de vendas atingiu o ponto mais baixo em 10 anos"
).configure_axis(
    grid=False
)

alt.Chart(...)

Os dados do Ipea confirmam a necessidade de soluções na área de varejo.

## Análise do setor pelo Google Trends

A falta de informações sobre os novos hábitos e perfil dos clientes pesa negativamente na tomada de decisão e da criação de novas alternativas para atingir o consumidor. Pensando nisso decidimos focar nossa solução nesses hábitos.

Utilizando o pytrends podemos acessar em código o google trends para obter dados de pesquisa de usuários, ganhando uma noção de interesse através dos termos mais pesquisados. 

Para ilustração dessa diferença de tendências de consumo pré pandemia fizemos uma pesquisa com termos do setor livreiro e de alimentação.

In [4]:
from pytrends.request import TrendReq

pytrend = TrendReq(hl='BR', tz=180)

In [5]:
palavras_chave = ['restaurante', 'ifood',
                  'livraria', 'kindle']

pytrend.build_payload(
     kw_list=palavras_chave,
     cat=0,
     timeframe='2019-01-01 2020-06-01',
     geo='BR',
     gprop='')

pytrend_data = pytrend.interest_over_time().drop(['isPartial'], axis=1)

alt.Chart(pytrend_data.reset_index()
          ).transform_fold(
    palavras_chave,
    ).mark_line().encode(
    x=alt.X('date:T',
            axis = alt.Axis(format=('%m/%y'),
            title = '')),
    y=alt.Y('value:Q', 
            title='Index de buscas no Google'),
    color=alt.Color('key:N', title='Termo',
                    sort=['restaurante', 'ifood', 'livraria', 'kindle'])
).properties(
    title='Em alguns setores houve inversão de algumas tendências durante a pandemia' 
).configure_axis(
    grid=False
)

alt.Chart(...)

Pensando em trabalhar com o parceiro do hackathon, o Santander, olhamos para o setor bancário para entendê-lo e procurar oportunidades conjuntas.

In [6]:
palavras_chave = ['financiamento',
            'financiamento santander',
            'santander', 'banco']

pytrend.build_payload(
     kw_list=palavras_chave,
     cat=0,
     timeframe='2019-01-01 2020-06-01',
     geo='BR',
     gprop='')

pytrend_data = pytrend.interest_over_time().drop(['isPartial'], axis=1)

alt.Chart(pytrend_data.reset_index(),
          title = ''
          ).transform_fold(
    palavras_chave,
    ).mark_line().encode(
    x=alt.X('date:T',
            title='',
            axis = alt.Axis(format=('%m/%y'))),
    y=alt.Y('value:Q', 
            title='Index de buscas no Google'),
    color=alt.Color('key:N', title='Termo',
                    sort=['banco', 'santander', 'financiamento'])
).properties(
    title='Serviços bancários estão em alta em 2020'
).configure_axis(
    grid=False
)


alt.Chart(...)

Os termos ligados a bancos tiveram um alta esse ano como consequência da pandemia.

Agora vamos olhar mais para uns termos em específico.





In [7]:
palavras_chave = ['linha de crédito',
            'crédito para empresas']

pytrend.build_payload(
     kw_list=palavras_chave,
     cat=0,
     timeframe='2019-01-01 2020-06-01',
     geo='BR',
     gprop='')

pytrend_data = pytrend.interest_over_time().drop(['isPartial'], axis=1)

alt.Chart(pytrend_data.reset_index(),
          title = ''
          ).transform_fold(
    palavras_chave,
    ).mark_line().encode(
    x=alt.X('date:T',
            title='',
            axis = alt.Axis(format=('%m/%y'))),
    y=alt.Y('value:Q', 
            title='Index de buscas no Google'),
    color=alt.Color('key:N', title='Termo',
                    sort=['linha de crédito'])
).properties(
    title='Pesquisas relacionadas a crédito cresceram durante a pandemia'
).configure_axis(
    grid=False
)

alt.Chart(...)

Agora que já vimos um geral do crescimento dos serviços bancários para empresas, podemos ver alguns produtos ainda mais de nicho.

O Santander possui alguns produtos direcionados a pequenos negócios, vimos 
como eles se comportaram no ultimo ano, com a curva de 'crédito para empresas' para comparação.

In [8]:
palavras_chave = ['santander prospera',
            'santander copiloto',
            'crédito para empresas']

pytrend.build_payload(
     kw_list=palavras_chave,
     cat=0,
     timeframe='2019-01-01 2020-06-01',
     geo='BR',
     gprop='')

pytrend_data = pytrend.interest_over_time().drop(['isPartial'], axis=1)

alt.Chart(pytrend_data.reset_index()
          ).transform_fold(
    palavras_chave,
    ).mark_line().encode(
    x=alt.X('date:T',
            axis = alt.Axis(format=('%m/%y'),
            title = '')),
    y=alt.Y('value:Q', 
            title='Index de buscas no Google'),
    color=alt.Color('key:N', title='Termo',
                    sort=['crédito para empresas', 'santander prospera'])
).properties(
    title='Copiloto e Prospera estão se popularizando mas há espaço de crescimento' 
).configure_axis(
    grid=False
)

alt.Chart(...)

O setor de crédito e financiamento para empresas está em alta mas as soluções Prospera e Copiloto ainda podem ganhar espaço e atingir mais pequenas empresas.

Decidimos então aproveitar essa oportunidade para criar uma solução em conjunto com esses negócios. Seria proposto um chatbot de recomendação de tendências do setor e análise de perfil do usuário, aproveitando a alta do setor bancário e que ele já está próximo do consumidor afetado.

Além de que o novo produto pode impulsionar os serviços já existentes e impactar a vida de mais empresários.